In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Assignment 3: Improving the Pipeline

## Things to fix in my pipeline
1. ~~A way to read different types of data.~~
2. ~~Make it so that the plotting functions have a title.~~
3. A pre-processing function that handles outliers.
4. ~~A function that discretizes continuous features,~~ and ~~one that turns a categorical variable into dummies.~~
5. ~~Add more classifiers to the pipeline~~
    * Added logit, KNN, decision trees, SVM, random forests, naive bayes, and gradient boostered trees.
6. ~~Experiment with different parameters for these classifiers.~~
7. ~~Add additional evaluation metrics.~~
8. ~~Create temporal validation function.~~ (sort of)

## Analysis

First, we need to pre-process the data set. We begin for filtering the data for the years we're interested in, 2011-2013. Then, we create the outcome labels we're interested in (whether or not a project got fully funded).

In [2]:
import pipeline_v2 as pxy

In [3]:
df = pxy.read_data("projects.csv", "csv")
ldf = pxy.read_data("outcomes.csv", "csv")
df['year_dt'] = pxy.create_datetime(df, 'date_posted')
df['year_posted'] = pxy.retrieve_year(df, 'year_dt')

In [10]:
# filter for data from 2011-2013
df2 = df.loc[(df['year_posted']>2010) & (df['year_posted']<2014),:]

In [28]:
def left_merge(df_left, df_right, merge_column):
    return pd.merge(df_left, df_right, how = 'left', on = merge_column)

In [35]:
maindata = left_merge(df2, ldf, "projectid")

In [36]:
maindata['true_labs'] = np.nan
for i in range(0, len(maindata)):
    if maindata.loc[i,'fully_funded'] == "t":
        maindata.loc[i,'true_labs'] = 1
    if maindata.loc[i,'fully_funded'] == "f":
        maindata.loc[i,'true_labs'] = 0

In [38]:
maindata['month_posted'] =pxy.retrieve_month(df, 'year_dt')

In [44]:
subdata = pxy.take_sample(maindata, 0.02).reset_index()

## Splitting the data into training and test sets:

First I split my data into the training set and test set. The training set consists of projects that were posted from 1/2011-6/2013, while the test set comprises projects from 7/2013 to 12/2013.

## Temporal cross-validation over the training set

I will also be performing temporal cross-validation over the training set, using the TimeSeriesSplit function from sklearn.

In [84]:
maindata = pxy.read_data("main_data.csv", "csv")

In [46]:
from datetime import datetime

In [57]:
traindata = maindata.loc[maindata['year_dt'] >= cutoff_dt,:]

In [85]:
prelim_featlist = ['teacher_teach_for_america', # dum
                   'school_charter', # dum
                   'school_magnet', # dum
                   'primary_focus_subject', # cat
                    'resource_type', # cat
                   'poverty_level', # cat
                   'grade_level', # cat
                   'total_price_excluding_optional_support', # cont
                   'students_reached' # cont
                  ]

## Feature generation

In [86]:
df_feat1 = maindata[prelim_featlist]

In [87]:
df_feat2 = pxy.generate_dummy(
    pxy.generate_dummy(pxy.generate_dummy(df_feat1, 'primary_focus_subject', attach= True),
    'poverty_level', attach = True),
    'grade_level', attach = True
)

In [88]:
df_feat3 = df_feat2.drop(columns = ['primary_focus_subject', 'poverty_level', 'grade_level'])

In [89]:
pd.set_option("display.max_columns",999)

In [90]:
# recode dummies to binary
for i in range(0,3):
    feat_name = prelim_featlist[i]
    for row_i in range(0, len(df_feat3)):
        if df_feat3.loc[row_i,prelim_featlist[i]] == "t":
            df_feat3.loc[row_i,prelim_featlist[i]] = 1
        else:
            df_feat3.loc[row_i,prelim_featlist[i]] = 0

In [108]:
df_features = pxy.generate_dummy(df_feat3, "resource_type", attach = True).drop(columns = ['resource_type'])

In [109]:
df_features.to_csv("df_features_done.csv")

In [ ]:
#### reset and reload df_features

In [136]:
df_feats = pd.read_csv("df_features_done.csv")
df_feats = df_feats.set_index("Unnamed: 0")

In [137]:
# i did write a function based on this afterwards
for i in range(0,46):
    df_feats.iloc[:,i] = df_feats.iloc[:,i].fillna(df_feats.iloc[:,i].mode()[0])

In [139]:
df_feats_final = pd.concat([df_feats, maindata['year_dt']], axis =1)

Form the main training and testing set:

In [142]:
cutoff_dt = datetime(2013, 7, 1, 0, 0)

In [147]:
df_feats_final['year_dt'] = pxy.create_datetime(df_feats_final ,'year_dt')
df_outcomes['year_dt'] = pxy.create_datetime(df_outcomes ,'year_dt')

'2013-12-31'

In [173]:
# don't drop dt column for training sets b/c we still need them ...
x_train = df_feats_final.loc[df_feats_final['year_dt'] < cutoff_dt,:]
x_test = df_feats_final.loc[df_feats_final['year_dt'] >= cutoff_dt,:].drop(columns = 'year_dt')

In [172]:
y_train = df_outcomes.loc[df_outcomes['year_dt'] < cutoff_dt,:]
y_test = df_outcomes.loc[df_outcomes['year_dt'] >= cutoff_dt,:].drop(columns = 'year_dt')

Create temporal cross validation

In [164]:
cv_time_thresholds = [2012, 2013]
cv_dict = {"iter_1":{}, "iter_2":{}}

In [207]:
xcv1_train = x_train.loc[x_train['year_dt'] < datetime(2012,1,1),:].drop(columns = 'year_dt')
xcv1_test = x_train.loc[(x_train['year_dt'] < datetime(2012,1,1)) & (x_train['year_dt'] < datetime(2013,1,1)),:].drop(columns = 'year_dt')
xcv2_train = x_train.loc[x_train['year_dt'] < datetime(2013,1,1),:].drop(columns = 'year_dt')
xcv2_test = x_train.loc[x_train['year_dt'] >= datetime(2013,1,1),:].drop(columns = 'year_dt')

In [208]:
ycv1_train = y_train.loc[y_train['year_dt'] < datetime(2012,1,1),:].drop(columns = 'year_dt')
ycv1_test = y_train.loc[(y_train['year_dt'] < datetime(2012,1,1)) & (y_train['year_dt'] < datetime(2013,1,1)),:].drop(columns = 'year_dt')
ycv2_train = y_train.loc[y_train['year_dt'] < datetime(2013,1,1),:].drop(columns = 'year_dt')
ycv2_test = y_train.loc[y_train['year_dt'] >= datetime(2013,1,1),:].drop(columns = 'year_dt')

In [116]:
model_params = {
    'max_features': ["auto"], 
    'n_estimators' : [5, 10, 20, 50, 100], 
    "max_depth": [3,5,8], 
    "criterion": ["gini", "entropy"]
    }

In [221]:
def custom_predictions(clf, threshold, x_test):
    # threshold = the probability threshold for something to be a 0.
    # generate array with predicted probabilities
    pred_array = clf.predict_proba(x_test)

    # initialize an empty array for the predictions
    pred_generated = np.array([])

    # predict the first entry
    if pred_array[0][0] >= threshold:
        pred_generated = np.hstack([pred_generated, 0])
    else:
        pred_generated = np.hstack([pred_generated, 1])

    # loops over the rest of the array
    for i in range(1,len(x_test)):
        if pred_array[i][0] >= threshold:
            pred_generated = np.vstack([pred_generated, 0])
        else:
            pred_generated = np.vstack([pred_generated, 1])

    # return an np.array
    return  pred_generated

In [222]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score

In [241]:
def run_cv_rf(threshold_list, model_params):
    results_rf = {}
    max_feats, num_dtrees_list, depth_list, criterion_list = model_params['max_features'], model_params['n_estimators'],  model_params["max_depth"], model_params["criterion"] 
    for threshold_i in threshold_list:
         for feature_number in max_feats:
            # loop over n_estimators
            for num_trees in num_dtrees_list:
                #loop over max_depth
                for depth_num in depth_list:
                    #loop over information gain criterion
                    for criterion_choice in criterion_list:
                        rf_clf = RandomForestClassifier(max_features = feature_number, n_estimators = num_trees, max_depth = depth_num, criterion = criterion_choice)
                        rf_clf.fit(xcv1_train, ycv1_train)
                        y_pred = custom_predictions(rf_clf, threshold = threshold_i, x_test = xcv1_test)
                        prec1 = accuracy_score(y_pred, ycv1_test)
                        rec1 = recall_score(y_pred, ycv1_test)
                        try:
                            roc1 = roc_auc_score(y_pred, ycv1_test)
                        except ValueError:
                            roc1 = 9999999
                        rf_clf = RandomForestClassifier(max_features = feature_number, n_estimators = num_trees, max_depth = depth_num, criterion = criterion_choice)
                        rf_clf.fit(xcv2_train, ycv2_train)
                        y_pred = custom_predictions(rf_clf, threshold = threshold_i, x_test = xcv2_test)
                        # intialize the main dictionary key
                        model_key = ("Random Forest", feature_number, num_trees, depth_num, criterion_choice, threshold_i)
                        prec2 = accuracy_score(y_pred, ycv2_test)
                        rec2 = recall_score(y_pred, ycv2_test)
                        try:
                            roc2 = roc_auc_score(y_pred, ycv2_test)
                        except ValueError:
                            roc2 = 9999999
                        
                        results_rf[model_key] = {}
                        # write evaluation results to dictionary
                        results_rf[model_key]['Precision']= (prec1 + prec2)/2
                        results_rf[model_key]['Recall']= (rec1 + rec2)/2
                        results_rf[model_key]['AUC Score'] = (roc1 + roc2)/2
    return results_rf

In [242]:
thresh_list = [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.5]

In [243]:
results_rf = run_cv_rf(thresh_list, model_params)

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launc

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics

...

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/xiangyu/anaconda3/lib

In [244]:
def run_cv_lr(threshold_list):
    results_lr = {}
    penalty_list = ['l1', 'l2']
    cvalues_list = [10**-5, 10**-2, 10**-1, 1, 10, 10**2, 10**5]
    max_feats, num_dtrees_list, depth_list, criterion_list = model_params['max_features'], model_params['n_estimators'],  model_params["max_depth"], model_params["criterion"] 
    for threshold_i in threshold_list:
        for c_para in cvalues_list:
            for penalty_para in penalty_list:
                #initialize logistic regression object with combination of parameters
                logreg = LogisticRegression(penalty = penalty_para, C = c_para)
                logreg.fit(xcv1_train, ycv1_train)
                y_pred = custom_predictions(logreg, threshold = threshold_i, x_test = xcv1_test)                

                prec1 = accuracy_score(y_pred, ycv1_test)
                rec1 = recall_score(y_pred, ycv1_test)
                try:
                    roc1 = roc_auc_score(y_pred, ycv1_test)
                except ValueError:
                    roc1 = 9999999                

                logreg = LogisticRegression(penalty = penalty_para, C = c_para)
                logreg.fit(xcv2_train, ycv2_train)
                y_pred = custom_predictions(logreg, threshold = threshold_i, x_test = xcv2_test)                

                prec2 = accuracy_score(y_pred, ycv2_test)
                rec2 = recall_score(y_pred, ycv2_test)
                try:
                    roc2 = roc_auc_score(y_pred, ycv2_test)
                except ValueError:
                    roc2 = 9999999      

                # intialize the main dictionary key
                model_key = ("LR", c_para, penalty_para, threshold_i)
                prec2 = accuracy_score(y_pred, ycv2_test)
                rec2 = recall_score(y_pred, ycv2_test)
                
                results_lr[model_key] = {}
                # write evaluation results to dictionary
                results_lr[model_key]['Precision']= (prec1 + prec2)/2
                results_lr[model_key]['Recall']= (rec1 + rec2)/2
                results_lr[model_key]['AUC Score'] = (roc1 + roc2)/2

    return results_lr

In [245]:
results_lr1 = run_cv_lr(thresh_list)

/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/xiangyu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [248]:
lr_df = pd.DataFrame(results_lr1)

In [249]:
rf_df = pd.DataFrame(results_rf)

# Report

Brief reflection: it took me quite some time to figure out how to do the temporal validation. I tried gridsearchcv and timeseriesplit to ultimately no avail. I hacked together my own cross validation function, but I'll have to think harder about constructing a generalizable function for my pipeline.

## Who are the best performers?

Let's assume that the baseline predictor guesses that every project gets fully funded. Such a predictor would have a precision of 0.7014 and recall of x, where x is the proportion of population we hve the resources to intervene on.

In [287]:
rf_df.transpose().sort_values("Precision", ascending = False).head(3)

AUC Score  Precision    Recall
Random Forest auto 5  8 gini    0.5  6.849798e-01   0.683784  0.683629
                   20 8 entropy 0.5  5.000000e+06   0.683127  0.682869
                   10 8 entropy 0.5  6.431536e-01   0.682823  0.682554

In [260]:
(247698/(105453+247698)

0.7013940212543642

In [265]:
lr_df.transpose().sort_values("Precision", ascending = False).head(3)

AUC Score  Precision    Recall
LR 100000.0 l2 0.5   0.630473   0.686536  0.690542
   10.0     l1 0.5   0.629073   0.686464  0.690620
   1.0      l1 0.5   0.629166   0.686447  0.690557

Comparing the top performers in terms of precision, we can see that that random forests with the parameters (max feat = 5, num trees = 5, depth = 8, gini) performs slightly worse than the best performing LR model (C = 1000000, L2 regularization).

In [288]:
rf_df.transpose().sort_values("Recall", ascending = False).head(3)

AUC Score  Precision    Recall
Random Forest auto 100 8 entropy 0.1   0.659133   0.318300  1.000000
                   50  8 entropy 0.1   0.653344   0.318458  0.988372
                   20  8 entropy 0.1   0.652575   0.318434  0.986842

In [289]:
lr_df.transpose().sort_values("Recall", ascending = False).head(3)

AUC Score  Precision    Recall
LR 0.01 l2 0.1   0.651898   0.319547  0.985149
        l1 0.1   0.644251   0.319805  0.969799
   0.10 l2 0.1   0.628379   0.321192  0.937656

On the other hand, RFs seem to do better at recall.

I didn't have the time to do a proper thorough analysis; I also need a clearer idea of what setting different thesholds means (is this the prob. threshold of a prediction? Or the threshold of population?)